Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import lightgbm as lgb
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
corpus = data['text']

In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
documents = []

In [9]:
stemmer = WordNetLemmatizer() # NLTK лемматизация

In [10]:
for sen in range(0, len(corpus)):
    document = re.sub(r'\W', ' ', str(corpus[sen])) # удалаяем все специальные сиволы
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) # удаляем все отдельные символы
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) # удаляем отдельные символы сначала
    document = re.sub(r'\s+', ' ', document, flags=re.I) # замена нескольких пробелов одним
    
    document = document.lower() # убераем заглавные буквы
    
    document = document.split() 
    # лематизация
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

In [11]:
y = data['toxic']

Разбиваем данные на тренировочную, валидационную и тестовую выборки.

In [12]:
X_train, X_valid_test, y_train, y_valid_test = train_test_split(documents, y, test_size=0.3, random_state=12345)

In [13]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test, y_valid_test, test_size=0.5, random_state=12345)

# 2. Обучение

Векторизация счетчика слов с учетом обработки стоп слов.

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
tfidf = TfidfVectorizer(max_features=5000, min_df=20, max_df=0.7, stop_words=stop_words)

In [16]:
X_train = tfidf.fit_transform(X_train)

## LGBMClassifier

In [17]:
gbm = lgb.LGBMClassifier(random_state=12345)

In [18]:
%%time
gbm.fit(X_train, y_train) # долгое время обучения

CPU times: user 2min 53s, sys: 662 ms, total: 2min 54s
Wall time: 2min 56s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
X_valid_gbm = tfidf.transform(X_valid)

In [20]:
y_pred_gbm = gbm.predict(X_valid_gbm)

In [21]:
f1_score(y_valid, y_pred_gbm) # не дотягивает до 0.75

0.7481499164478395

## RandomForestClassifier

In [22]:
%%time
for num in range(300, 350, 50):
    print('max_depth', num)
    for rate in range(140, 160, 10):
        print('n_estimators:', rate)
        model_RFC = RandomForestClassifier(max_depth = num, n_estimators = rate, random_state=12345)
        model_RFC.fit(X_train, y_train)
        X_valid_v_RFC = tfidf.transform(X_valid)
        pred_valid_RFC = model_RFC.predict(X_valid_v_RFC)
        print('F1:', f1_score(y_valid, pred_valid_RFC))
    print()
print()

max_depth 300
n_estimators: 140
F1: 0.7578413284132841
n_estimators: 150
F1: 0.7594644506001845


CPU times: user 13min 5s, sys: 185 ms, total: 13min 6s
Wall time: 13min 7s


Метрика F1 больше 0.75. Можно приступить к проверке на тестовой выборке.

### Тестирование

In [23]:
model = RandomForestClassifier(max_depth = 300, n_estimators = 140, random_state=12345)

In [24]:
%%time
model.fit(X_train, y_train)

CPU times: user 6min 33s, sys: 322 ms, total: 6min 34s
Wall time: 6min 36s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=300, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=140,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [25]:
X_test_v = tfidf.transform(X_test)

In [26]:
pred_test = model.predict(X_test_v)

In [27]:
f1_score(y_test, pred_test)

0.7505773672055428

## LogisticRegression

In [28]:
model_LR = LogisticRegression(random_state=12345)

In [29]:
model_LR.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
X_valid_LR = tfidf.transform(X_valid)

In [31]:
pred_LR = model_LR.predict(X_valid_LR)

In [32]:
f1_score(y_valid, pred_LR) # не дотягивает до 0.75

0.7465886939571151

# 3. Выводы

В результате сравнения различных моделей наиболее эфективная модель, способная достичь значений метрики f1 не менее 0.75 будет:
RandomForestClassifier с параметарми max_depth = 300 и n_estimators = 140.
Превод тектовой информации в векторный вид и обучение таких моделей довольно трудоемкие процессы.